In [20]:
# Build a formula evaluator

In [21]:
import pickle

def load_pickled_object(file_path):
    with open(file_path, 'rb') as file:
        obj = pickle.load(file)
    return obj

In [22]:
generic_formula_dictionary = load_pickled_object('generic_formula_dictionary.pkl')
series_dict = load_pickled_object('series_dict.pkl')

In [27]:
series_dict["Sheet1"]

[Series(series_id=SeriesId(sheet_name='Sheet1', series_header='col_1', series_header_cell_row=2, series_header_cell_column=2), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='col_1', formulas=[None, None], values=[1, 3], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=2, row=3, coordinate='B3', sheet_name=None, value=None, value_type=None, formula=None), series_length=2, series_data_type=<SeriesDataType.INT: 'int'>),
 Series(series_id=SeriesId(sheet_name='Sheet1', series_header='col_2', series_header_cell_row=2, series_header_cell_column=3), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='col_2', formulas=[None, None], values=[2, 4], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=3, row=3, coordinate='C3', sheet_name=None, value=None, value_type=None, formula=None), series_length=2, series_data_type=<SeriesDataType.INT: 'int'>)

In [23]:
example_ast = list(generic_formula_dictionary.values())[0]

In [ ]:
# What do I need: a dictionary with Series Ids as keys and pandas Series as values (accounting for the orientation)

import pandas as pd

series_dictionary_simple = {"Sheet1|col_1|2|2": pd.Series([1, 2, 3, 4, 5]), "Sheet1|col_2|2|3": pd.Series([6, 7, 8, 9, 10])}

In [24]:
print(example_ast)

((('Sheet1|col_1|2|2',), (0, 0), (1, 1), (0, 0))) + ((('Sheet1|col_2|2|3',), (0, 0), (1, 1), (0, 0)))
